In [2]:
import sys
import pandas as pd
from pathlib import Path
from IPython.display import clear_output
import dask
from dask import dataframe as dd
import numpy as np
import tensorflow as tf
import bctpy

ModuleNotFoundError: No module named 'bctpy'

In [ ]:
pconncsv_dir = Path('/fast_scratch/jdr/ABCD/ABCD_3165_Pipe/pconns_working/pconn_csv')
subjIDs = []
allArrays = []

for pconncsv in pconncsv_dir.glob('*.csv'):
    clear_output(wait=True)
    subjIDs.append(pconncsv.name.split('_')[0])
    
    csv_in_sq = pd.read_csv(pconncsv, sep="\t", header=None).values
    csv_in_flat = csv_in_sq[np.tril_indices(453, k = -1)]
    
    if len(allArrays) == 0:
        allArrays = csv_in_flat
    else:
        allArrays = np.vstack((allArrays, csv_in_flat))
    print(pconncsv)
    
allArraysPath = Path('/fast_scratch/jdr/ABCD/ABCD_3165_Pipe/Data/allArrays.npy')
np.save(allArraysPath, allArrays)

subjIDs_arrayPath = Path('/fast_scratch/jdr/ABCD/ABCD_3165_Pipe/Data/subjIDs.npy')
np.save(subjIDs_arrayPath, np.asarray(subjIDs))

In [ ]:
c_data = pd.read_table('/fast_scratch/jdr/ABCD/ABCD_3165_Pipe/Data/abcd_ptsd01.txt', header=0, skiprows=[1])
c_data = c_data[c_data["eventname"] == "baseline_year_1_arm_1"]
c_data = c_data.drop(['collection_id', 'abcd_ptsd01_id', 'dataset_id', 
                      'src_subject_id', 'interview_date', 'eventname', 'collection_title'], axis=1)
c_data['subjectkey'] = 'sub-' + c_data['subjectkey'].str.replace('_', '')
c_data = c_data.drop(['ExposureTotal', 'IPVTotal'], axis=1, errors='ignore')
c_data['ExposureTotal'] = c_data.iloc[:, 3:19].sum(axis=1)
c_data['IPVTotal'] = c_data.iloc[:, 7:18].sum(axis=1)
c_data['ExposureAny'] = np.where(c_data['ExposureTotal'] == 0, 0, 1)
c_data['IPVAny'] = np.where(c_data['IPVTotal'] == 0, 0, 1)
c_data['Sex01'] = np.where(c_data['sex'] == 'M', 1, 0) # Must use numeric data as input (not M, F)

In [ ]:
subjIDs_df = pd.DataFrame(subjIDs,columns=['subjectkey'])
 = subjIDs_df.merge(c_data, how='left', on='subjectkey')
df_comb.info()

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(allArrays, df_comb['Sex01'].values)

In [ ]:

subjIDs

In [ ]:
#Creating the classification model
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

classifier = Sequential()
#First Hidden Layer
classifier.add(Dense(32, activation='tanh', kernel_initializer='random_normal', input_shape=x_train.shape[1:]))
#Second  Hidden Layer
classifier.add(Dense(16, activation='relu', kernel_initializer='random_normal'))
#Third  Hidden Layer
classifier.add(Dense(16, activation='relu', kernel_initializer='random_normal'))
#Output Layer
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))

In [ ]:
classifier.compile(optimizer = Adam(lr =.0001),loss='binary_crossentropy', metrics =['accuracy'])

In [ ]:
classifier.fit(x_train_tf,y_train_recode, batch_size=32, epochs=100)